In [1]:
import plotly.graph_objects as go
import numpy as np
from nptyping import NDArray, Shape, Float, Int
import plotly.express as px

def calculate_vertices(xmin=0, ymin=0, zmin=0, xmax=None, ymax=None, zmax=None):
    xmax = xmin + 1 if xmax is None else xmax
    ymax = ymin + 1 if ymax is None else ymax
    zmax = zmin + 1 if zmax is None else zmax
    return {
        "x": [xmin, xmin, xmax, xmax, xmin, xmin, xmax, xmax],
        "y": [ymin, ymax, ymax, ymin, ymin, ymax, ymax, ymin],
        "z": [zmin, zmin, zmin, zmin, zmax, zmax, zmax, zmax],
        "i": [7, 0, 0, 0, 4, 4, 6, 1, 4, 0, 3, 6],
        "j": [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
        "k": [0, 7, 2, 3, 6, 7, 1, 6, 5, 5, 7, 2],
    }

def draw_cube(xmin=0, ymin=0, zmin=0, xmax=None, ymax=None, zmax=None, color = "gold", opacity = 0.5, name = "cube", colors_shown_in_legend = set()):
    vertices = calculate_vertices(xmin, ymin, zmin, xmax, ymax, zmax)
    if color in colors_shown_in_legend:
        showlegend = False
    else:
        showlegend = True
        colors_shown_in_legend.add(color)
    fig = go.Figure(data=[
        go.Mesh3d(
            # 8 vertices of a cube
            x=vertices['x'],
            y=vertices['y'],
            z=vertices['z'],
            colorscale=[[0, 'gold']],
            # i, j and k give the vertices of triangles
            i = vertices["i"],
            j = vertices["j"],
            k = vertices["k"],
            color = color,
            showscale=True,
            opacity = opacity,
            name=name,
            showlegend=showlegend,
            legendgroup = color
        )
    ])
    return fig, colors_shown_in_legend


def sum_figures(fig_list):
    fig = go.Figure()
    for f in fig_list:
        fig.add_traces(f.data)
    return fig

def coordinates_to_grid_indices(coordinates: NDArray[Shape["NPixels, 3"], Float], 
                                grid_index_of_origin: NDArray[Shape["3"], Int], 
                                voxel_size: float) -> NDArray[Shape["NPixels, 3"], Int]:
        return (np.floor(coordinates/voxel_size) + grid_index_of_origin).astype(int)

def grid_indices_to_world_coordinates(grid_indices: NDArray[Shape["NPixels, 3"], Int], 
                                      grid_index_of_origin: NDArray[Shape["3"], Int], 
                                      voxel_size: float, 
                                      voxel_center_coordinates: bool = False) -> NDArray[Shape["NPixels, 3"], Float]:
    # Returns the origin of the voxel
    voxel_origin_world_coordinates = (grid_indices - grid_index_of_origin)*voxel_size
    if voxel_center_coordinates:
        voxel_origin_world_coordinates += voxel_size/2
    return voxel_origin_world_coordinates


def coordinates_to_grid_indices(coordinates: NDArray[Shape["NPixels, 3"], Float],
                                grid_index_of_origin: NDArray[Shape["3"], Int],
                                voxel_size: float) -> NDArray[Shape["NPixels, 3"], Int]:
    return (np.floor(coordinates/voxel_size) + grid_index_of_origin).astype(int)

In [ ]:
origin_position = (20,20,20)
_RESOLUTION = 1.0  # cm per pixel
_EGOCENTRIC_MAP_SHAPE = (500, 500, 500)  # (x, y, z) in pixel
_EGOCENTRIC_MAP_ORIGIN_POSITION = (250, 250, 250)  # (x, y, z) in pixel
_NUM_SEMANTIC_CLASSES = 10


a = [((0,0), (0,0,1)), 
     ((0,1), (0,0,2)),
     ((1,0), (0,0,3)),
     ((1,1), (0,0,4))]

coords_2d = np.array([x[0] for x in a])
coords_3d = np.array([x[1] for x in a])
semantic_map = np.random.rand(2,2,_NUM_SEMANTIC_CLASSES)

grid_indices = coordinates_to_grid_indices(coords_3d, _EGOCENTRIC_MAP_ORIGIN_POSITION,
                                            _RESOLUTION)

egocentric_occupancy_map = np.zeros(shape = _EGOCENTRIC_MAP_SHAPE + (_NUM_SEMANTIC_CLASSES+1, ),
                                    dtype = np.float32)
egocentric_occupancy_map[grid_indices[:,0], grid_indices[:,1], grid_indices[:,2], 0] = 1
egocentric_occupancy_map[grid_indices[:,0], grid_indices[:,1], grid_indices[:,2], 1:] = (
    semantic_map[coords_2d[:,0], coords_2d[:,1]])
grid_indices
semantic_map[0,0,:]
egocentric_occupancy_map[250,250,251]

indices = np.array(np.where(egocentric_occupancy_map[:,:,:,0])).transpose()
semantic_classes = np.argmax(egocentric_occupancy_map[np.where(egocentric_occupancy_map[:,:,:,0])][:,1:], axis=1)
cubes = []
colors_shown_in_legend = set()
colors = px.colors.sample_colorscale("viridis", [n/(10 -1) for n in range(10)])

for i, (x,y,z) in enumerate(grid_indices_to_world_coordinates(indices, _EGOCENTRIC_MAP_ORIGIN_POSITION, _RESOLUTION)):
    cube, colors_shown_in_legend = draw_cube(x,y,z, color = colors[semantic_classes[i]], opacity = 0.5, name = str(semantic_classes[i]), colors_shown_in_legend=colors_shown_in_legend)
    cubes.append(cube)

fig = go.Figure()
for f in cubes:
    fig.add_traces(f.data)
    
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=2, y=2, z=2)
)

fig.update_layout(
    autosize=False,
    width=500,
    height=500, 
    margin=dict(l=0, b=0),
    scene_camera = camera,
    template = "plotly_dark",
    showlegend=True,
    legend_title_text='Semantic category' #Update Legend header if you dont like 'variable'
    )
fig.update_layout(title="Try Clicking on the Legend Items!")
fig.show()

NameError: name 'coordinates_to_grid_indices' is not defined